In [1]:
#Import Utilities for Datathon
from DataLoader import *
from ScheduleVisualizer import *
from CheckSchedule import checkSchedule
from satUtils import *

# Import Helpful packages
import pandas as pd
import pprint
import json

# Helpful to print out dictionaries
pp = pprint.PrettyPrinter(indent=4)

import copy

### Clean up pilots leave

In [2]:
# load and view data
pilots = loadPilotData()
print(len(pilots.keys()), 'total')
pilots[10000]

106 total


{'Leave': [[128.0, 135.0], [143.0, 143.0], [146.0, 146.0], [147.0, 208.0]],
 'Quals': {'Aug': 'EP A',
  'Sep': 'EP A',
  'Oct': None,
  'Nov': None,
  'Dec': None}}

#### Show invalid leave dates

In [3]:
# returns list of pilots who have invalid leave dates (i.e. negative return dates)
def get_invalid_leave(pilots):
    invalid_leave = {}
    for p in pilots.keys():
        for l in pilots[p]['Leave']:
            for day in l:
                if day < 0:
                    invalid_leave[p] = pilots[p]
    return invalid_leave
        
invalid_leave = get_invalid_leave(pilots)
print(len(invalid_leave), 'pilots with invalid leave')
for p in invalid_leave.keys():
    print('Pilot', p, ', Leave', invalid_leave[p]['Leave'])

3 pilots with invalid leave
Pilot 10029 , Leave [[202.0, 204.0], [218.0, -93.0], [181.0, 195.0]]
Pilot 10039 , Leave [[78.0, 86.0], [132.0, 141.0], [215.0, -108.0]]
Pilot 10084 , Leave [[192.0, 201.0], [145.0, 149.0], [218.0, -93.0], [202.0, 204.0]]


#### Clean pilot data to fix invalid leave

In [4]:
# clean up by setting negative values to be 1 day of leave
# note: assumes problem is only with end dates, not start dates
def clean_invalid_leave(pilots):
    invalid_leave = get_invalid_leave(pilots)
    for p in invalid_leave: 
        for i, l in enumerate(pilots[p]['Leave']):
            for j, day in enumerate(l):
                if day < 0:
                    if j == 1:  
                        pilots[p]['Leave'][i][j] = l[j-1]
    return pilots

pilots = clean_invalid_leave(pilots)

In [5]:
get_invalid_leave(pilots)

{}

### Clean up aircrew quals
Cleaning up according to Chris Chin's suggestion on Slack  
Ask Travis: is this only in the pilot data, not in events?

In [6]:
events = loadEventData()
events['f1']

{'Category': 'Local',
 'Type': 'Mission',
 'CrewType': 'Augmented',
 'CrewRequirements': ['MP+', 'FPQ+', 'FPC+'],
 'CrewMin': 3,
 'CrewMax': 3,
 'StartDay': 60.0,
 'EndDay': 65.0}

In [7]:
all_quals = []
for e in events.keys():
    all_quals += events[e]['CrewRequirements']
all_quals = set(all_quals)
all_quals

# are these really the only quals in the events?

{'',
 'EP+',
 'FPC+',
 'FPCCR+',
 'FPL+',
 'FPNC',
 'FPQ+',
 'IP AL+',
 'IP+',
 'MP C3+',
 'MP CJ+',
 'MP+'}

In [8]:
pilots[10000]

{'Leave': [[128.0, 135.0], [143.0, 143.0], [146.0, 146.0], [147.0, 208.0]],
 'Quals': {'Aug': 'EP A',
  'Sep': 'EP A',
  'Oct': None,
  'Nov': None,
  'Dec': None}}

In [9]:
quals_map = {'IPA2': 'IP A2',
             'FPKN': 'FPKA',
             'UP A': 'IP A',
             'MPCR': 'MP CR',
             'IPBZ': 'IP BZ'}

def get_invalid_pilot_quals(pilots, quals_map=quals_map):
    invalid_quals = {}
    for p in pilots.keys():
        for month in pilots[p]['Quals'].keys():
            if pilots[p]['Quals'][month]:
                qual = pilots[p]['Quals'][month]
                if qual in quals_map.keys():
                    invalid_quals[p] = pilots[p]  
    return invalid_quals

invalid_quals = get_invalid_pilot_quals(pilots)
print(len(invalid_quals), 'pilots with invalid quals')
for p in invalid_quals.keys():
    print('Pilot', p, ', Quals', invalid_quals[p]['Quals'])

6 pilots with invalid quals
Pilot 10005 , Quals {'Aug': 'IP A2', 'Sep': 'IPA2', 'Oct': 'IP A2', 'Nov': 'IPA2', 'Dec': 'IP A2'}
Pilot 10040 , Quals {'Aug': 'IPBZ', 'Sep': 'IP BZ', 'Oct': 'IP BJ', 'Nov': 'IP BJ', 'Dec': 'IP BJ'}
Pilot 10086 , Quals {'Aug': 'MPCR', 'Sep': 'MPCR', 'Oct': 'MP CR', 'Nov': 'MP CR', 'Dec': 'MP CZ'}
Pilot 10082 , Quals {'Aug': None, 'Sep': None, 'Oct': 'UP A', 'Nov': 'UP A', 'Dec': 'UP A'}
Pilot 10094 , Quals {'Aug': None, 'Sep': None, 'Oct': None, 'Nov': 'FPKA', 'Dec': 'FPKN'}
Pilot 10047 , Quals {'Aug': None, 'Sep': None, 'Oct': None, 'Nov': 'FPKN', 'Dec': 'FPKC'}


In [10]:
def clean_pilot_quals(pilots, quals_map=quals_map):
    for p in pilots.keys():
        for month in pilots[p]['Quals'].keys():
            if pilots[p]['Quals'][month]:
                qual = pilots[p]['Quals'][month]
                if qual in quals_map.keys():
                    pilots[p]['Quals'][month] = quals_map[qual]
    return pilots

pilots = clean_pilot_quals(pilots)

In [11]:
get_invalid_pilot_quals(pilots)

{}